In [1]:
from classical_music import *

file_txt = "100-classical-masterworks.txt"

file_html = file_txt.replace(".txt", ".html")

In [2]:
namemap = read_namemap()

In [3]:
meta_map, lines = parse_file_txt(file_txt)

meta_map, lines[:2]

({'vid': 'mVju10GtMXI',
  'vid_name': '100 Classical Masterworks',
  'channel_id': 'UClScm1QV2xecmZrAuADnP9g',
  'channel_name': 'Essential Classic'},
 ['', '### CLASSICAL PERIOD'])

In [4]:
# this is data file specific
def parse_line(l):
    """parse text-line into data elements
    
    sample:
        1 Mozart - Eine Kleine Nachtmusik: I. Allegro 00:00

    Returns:
        [id, ts, lastname, title]
    """
    tmp = l.split(" - ")
    tmp1 = tmp[0].split(" ")
    _id, lastname = tmp1[0], " ".join(tmp1[1:])
    
    tmp2 = " - ".join(tmp[1:]).split(" ")
    title = " ".join(tmp2[:-1])
    ts = tmp2[-1]
    return [_id.strip(), ts.strip(), lastname.strip(), title.strip()]


In [5]:
line = "1 Mozart - Eine Kleine Nachtmusik: I. Allegro 00:00"
print(parse_line(line))

['1', '00:00', 'Mozart', 'Eine Kleine Nachtmusik: I. Allegro']


In [6]:
records = []
for i in lines:
    i = i.strip()
    if not i: 
        continue
        
    # handle header
    if "###" in i:
        records.append([i])
        continue
        
    try:
        _id, ts, lastname, title = parse_line(i)
        if lastname not in namemap:
            print(f"[ERROR] {lastname} not found in namemap\n row={i}")
        else:
            records.append([_id, 
                            make_youtube_link(meta_map["vid"], ts), 
                            make_href(namemap[lastname]["fullname"] ,namemap[lastname]["url"]), 
                            title]
                        )
    except Exception as ex:
        raise ValueError(f"[ERROR] str(ex)\n row={i}")

In [7]:
records[:3]

[['### CLASSICAL PERIOD'],
 ['1',
  '<a href=https://www.youtube.com/watch?v=mVju10GtMXI&t=0s target=new>00:00</a>',
  '<a href=https://www.wikiwand.com/en/Wolfgang_Amadeus_Mozart target=new>Wolfgang Amadeus Mozart</a>',
  'Eine Kleine Nachtmusik: I. Allegro'],
 ['2',
  '<a href=https://www.youtube.com/watch?v=mVju10GtMXI&t=362s target=new>06:02</a>',
  '<a href=https://www.wikiwand.com/en/Ludwig_van_Beethoven target=new>Ludwig van Beethoven</a>',
  'Symphony No. 3, Op. 55 ‘Eroica’: I. Allegro con brio']]

In [9]:
# write out HTML file
with open(file_html, "w") as f:
    
    f.write(f"""
        <h1>
        <a href=https://www.youtube.com/watch?v={meta_map["vid"]}>{meta_map["vid_name"]}</a> </h1> 
        <h2>by <a href=https://www.youtube.com/channel/{meta_map["channel_id"]}>{meta_map["channel_name"]}</a>
        </h2>
        <br>
    """)

    h3_count = 0
    for r in records:
        # f.write(f"r = {r}\n")
        if len(r) == 1:
            
            if h3_count:
                f.write('</table>')                

            f.write(f"""
                <h3>{r[0].replace("###", "")}</h3>
                <table style="width:100%" border=1px>
                  <tr>
                    <th>Id</th>
                    <th>Track TS</th>
                    <th>Musician</th>
                    <th>Title</th>
                  </tr>
            """)
            
            h3_count += 1

        elif len(r) == 4:
            
            f.write(f"""<tr>
                <td>{r[0]}</td>
                <td>{r[1]}</td>
                <td>{r[2]}</td>
                <td>{r[3]}</td>
              </tr>
            """)
            
        else:
            print(f"[ERROR] unknown record {r}")
            
    f.write('</table>')